# 1. Introduction

In this notebook, we will create a proof-on-concept web application that shows the finally selected model in accion. The application will be created using the [Gradio](https://www.gradio.app/) framework and then deployed to HuggingFace to make it publicly available for all.

# 2. Importing Libraries

In [5]:
import os
import json
from pathlib import Path
from helper_functions import create_dataloaders, set_seeds

# 3. Creating Directories and Files

In [2]:
# Create FoodVision demo path
food_vision_demo_path = Path("../demo")  # Path joining with Path objects
food_vision_demo_path.mkdir(parents=True, exist_ok=True)

# Make examples directory
food_vision_demo_examples_path = food_vision_demo_path / "examples"
food_vision_demo_examples_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# Create a JSON file with the description of th 101 types of food
food_descriptions = {
    "apple_pie": "A classic American dessert made with spiced apple filling encased in a flaky pastry crust, often served with ice cream.",
    "baby_back_ribs": "Juicy, tender pork ribs, slow-cooked and coated with a flavorful barbecue sauce.",
    "baklava": "A rich, sweet pastry layered with chopped nuts and honey or syrup, originating from the Middle East and Mediterranean regions.",
    "beef_carpaccio": "Thinly sliced raw beef served with olive oil, lemon, capers, and Parmesan, often as a starter.",
    "beef_tartare": "Finely chopped or ground raw beef mixed with seasonings like capers, onions, and egg yolk, served uncooked.",
    "beet_salad": "A refreshing dish featuring cooked or raw beets paired with greens, goat cheese, nuts, and a tangy vinaigrette.",
    "beignets": "Deep-fried dough pastries dusted with powdered sugar, popular in New Orleans as a sweet treat.",
    "bibimbap": "A Korean mixed rice dish topped with vegetables, meat, an egg, and spicy gochujang sauce.",
    "bread_pudding": "A comforting dessert made from bread soaked in a sweet custard mixture and baked until golden.",
    "breakfast_burrito": "A hearty wrap filled with scrambled eggs, cheese, meats, and sometimes vegetables or beans.",
    "bruschetta": "An Italian appetizer of toasted bread rubbed with garlic and topped with fresh tomatoes, olive oil, and basil.",
    "caesar_salad": "A classic salad made with romaine lettuce, croutons, Parmesan cheese, and Caesar dressing.",
    "cannoli": "An Italian dessert consisting of a crispy pastry shell filled with sweet ricotta cheese and sometimes chocolate chips.",
    "caprese_salad": "A simple Italian salad of fresh mozzarella, tomatoes, and basil, drizzled with olive oil and balsamic vinegar.",
    "carrot_cake": "A moist cake made with grated carrots and often topped with cream cheese frosting.",
    "ceviche": "A seafood dish where fresh fish or shrimp is 'cooked' in citrus juice and mixed with onions, peppers, and cilantro.",
    "cheese_plate": "An assortment of cheeses served with accompaniments like crackers, fruit, and nuts.",
    "cheesecake": "A creamy dessert made with a mixture of cream cheese, sugar, and eggs on a graham cracker or pastry crust.",
    "chicken_curry": "A flavorful dish of chicken cooked in a spicy, aromatic curry sauce, popular in South Asian cuisine.",
    "chicken_quesadilla": "A Mexican dish of tortillas filled with cheese, chicken, and spices, grilled until crispy and melty.",
    "chicken_wings": "Juicy chicken wings fried or baked and often coated with a tangy or spicy sauce.",
    "chocolate_cake": "A rich, moist dessert made with layers of chocolate sponge and frosting.",
    "chocolate_mousse": "A light and creamy dessert made with whipped cream and melted chocolate, often served chilled.",
    "churros": "Fried dough pastry, often coated in sugar and cinnamon, typically served with chocolate dipping sauce.",
    "clam_chowder": "A creamy soup made with clams, potatoes, and onions, often served in a bread bowl.",
    "club_sandwich": "A layered sandwich filled with turkey, bacon, lettuce, tomato, and mayonnaise.",
    "crab_cakes": "Pan-fried or baked patties made from lump crab meat and breadcrumbs, seasoned with spices.",
    "creme_brulee": "A custard dessert with a caramelized sugar crust on top, served cold.",
    "croque_madame": "A French grilled sandwich filled with ham and cheese, topped with a fried egg.",
    "cup_cakes": "Small, individual-sized cakes often topped with frosting and decorations.",
    "deviled_eggs": "Hard-boiled eggs halved and filled with a seasoned yolk mixture.",
    "donuts": "Sweet, fried dough pastries, often glazed or topped with sprinkles.",
    "dumplings": "Small dough parcels filled with meat or vegetables, steamed, boiled, or fried.",
    "edamame": "Boiled or steamed young soybeans served in their pods, sprinkled with salt.",
    "eggs_benedict": "A breakfast dish of poached eggs, English muffins, Canadian bacon, and hollandaise sauce.",
    "escargots": "Cooked snails, often served with garlic butter in French cuisine.",
    "falafel": "Deep-fried balls or patties made from ground chickpeas or fava beans, often served in pita bread.",
    "filet_mignon": "A tender and juicy cut of beef steak, often served as a fine dining dish.",
    "fish_and_chips": "A classic British dish of battered and fried fish served with crispy fries.",
    "foie_gras": "A French delicacy made from the liver of a fattened duck or goose, often served as a pâté.",
    "french_fries": "Thinly sliced potatoes deep-fried until crispy, often served as a side dish.",
    "french_onion_soup": "A hearty soup made with caramelized onions and beef broth, topped with bread and melted cheese.",
    "french_toast": "Bread slices soaked in a milk and egg mixture, fried until golden, and served with syrup.",
    "fried_calamari": "Lightly battered and fried squid rings, often served with marinara or aioli sauce.",
    "fried_rice": "A stir-fried rice dish mixed with vegetables, eggs, and meat or seafood.",
    "frozen_yogurt": "A creamy, tangy dessert similar to ice cream but made with yogurt.",
    "garlic_bread": "Toasted bread slices flavored with garlic butter, often served as a side dish.",
    "gnocchi": "Soft Italian dumplings made from potato, flour, and sometimes cheese, served with sauce.",
    "greek_salad": "A fresh salad made with cucumbers, tomatoes, olives, feta cheese, and olive oil.",
    "grilled_cheese_sandwich": "A hot sandwich filled with melted cheese, grilled until crispy.",
    "grilled_salmon": "Salmon fillets cooked on a grill, often seasoned with herbs and spices.",
    "guacamole": "A creamy dip made from mashed avocados, lime juice, onions, and cilantro.",
    "gyoza": "Japanese dumplings filled with minced meat and vegetables, pan-fried and steamed.",
    "hamburger": "A ground beef patty served in a bun with toppings like lettuce, tomato, and cheese.",
    "hot_and_sour_soup": "A Chinese soup with a tangy and spicy flavor, filled with tofu, mushrooms, and vegetables.",
    "hot_dog": "A grilled or boiled sausage served in a bun with condiments like ketchup and mustard.",
    "huevos_rancheros": "A Mexican breakfast dish with fried eggs, tortillas, and salsa.",
    "hummus": "A creamy dip made from blended chickpeas, tahini, lemon juice, and garlic.",
    "ice_cream": "A frozen dessert made from sweetened cream, often flavored and served in scoops.",
    "lasagna": "An Italian pasta dish layered with meat, cheese, and tomato sauce, baked until bubbly.",
    "lobster_bisque": "A creamy soup made from lobster stock, often served as a gourmet dish.",
    "lobster_roll_sandwich": "A sandwich filled with chunks of lobster meat, dressed in butter or mayo.",
    "macaroni_and_cheese": "A comfort food dish of macaroni pasta mixed with a creamy cheese sauce.",
    "macarons": "Delicate French almond cookies filled with flavored buttercream or ganache.",
    "miso_soup": "A Japanese soup made with fermented soybean paste, tofu, and seaweed.",
    "mussels": "Shellfish steamed and served in a flavorful broth, often with bread.",
    "nachos": "Tortilla chips topped with melted cheese and various toppings like jalapeños and sour cream.",
    "omelette": "Beaten eggs cooked in a frying pan, often filled with cheese, meats, or vegetables.",
    "onion_rings": "Crispy, deep-fried onion slices, often served as a side dish or snack.",
    "oysters": "Shellfish often served raw on the half shell with lemon or hot sauce.",
    "pad_thai": "Stir-fried rice noodles with eggs, tofu or shrimp, peanuts, and a tamarind-based sauce.",
    "paella": "Spanish rice dish with seafood, meats, and saffron, traditionally cooked in a single pan.",
    "pancakes": "Fluffy round cakes made from flour, eggs, and milk, served with syrup and butter.",
    "panna_cotta": "Italian dessert of sweetened cream thickened with gelatin, often topped with berries or caramel.",
    "peking_duck": "Crispy duck served with pancakes, hoisin sauce, and sliced scallions.",
    "pho": "Vietnamese noodle soup with broth, rice noodles, herbs, and beef or chicken.",
    "pizza": "Flat dough topped with tomato sauce, cheese, and various toppings, baked until crispy.",
    "pork_chop": "Grilled or roasted cut of pork, often served with vegetables or a flavorful sauce.",
    "poutine": "Canadian dish with French fries, cheese curds, and gravy.",
    "prime_rib": "Tender beef rib roast, often seasoned and slow-cooked, served for special occasions.",
    "pulled_pork_sandwich": "Slow-cooked shredded pork with barbecue sauce, served on a bun.",
    "ramen": "Japanese noodle soup with broth, wheat noodles, and toppings like eggs and pork.",
    "ravioli": "Italian dumplings filled with cheese, meat, or vegetables, served with sauce.",
    "red_velvet_cake": "Rich cake with a hint of cocoa and red coloring, layered with cream cheese frosting.",
    "risotto": "Italian creamy rice dish cooked with broth and flavored with vegetables or meat.",
    "samosa": "Deep-fried pastry filled with spiced potatoes, peas, and sometimes meat.",
    "sashimi": "Thinly sliced raw fish or seafood, often served with soy sauce and wasabi.",
    "scallops": "Delicate shellfish, often seared and served as a seafood entrée.",
    "seaweed_salad": "Salad made from seasoned edible seaweed, typically with sesame oil and soy sauce.",
    "shrimp_and_grits": "Southern dish of shrimp cooked in a savory sauce, served with creamy grits.",
    "spaghetti_bolognese": "Italian pasta with a rich meat sauce made from ground beef, tomatoes, and herbs.",
    "spaghetti_carbonara": "Pasta with a creamy sauce made from eggs, cheese, pancetta, and black pepper.",
    "spring_rolls": "Rice paper rolls filled with vegetables, meat, or seafood, served with dipping sauce.",
    "steak": "Grilled or pan-seared beef cut, often served with vegetables or potatoes.",
    "strawberry_shortcake": "Dessert with layers of cake, fresh strawberries, and whipped cream.",
    "sushi": "Japanese dish with vinegared rice, raw or cooked seafood, and sometimes vegetables.",
    "tacos": "Mexican dish with a tortilla filled with meat, vegetables, cheese, and salsa.",
    "takoyaki": "Japanese snack made from batter, octopus, and tempura bits, served with takoyaki sauce.",
    "tiramisu": "Italian dessert with coffee-soaked ladyfingers, mascarpone cheese, and cocoa powder.",
    "tuna_tartare": "Finely diced raw tuna, often mixed with soy sauce and served as an appetizer.",
    "waffles": "Batter-based dish cooked in a grid pattern, served with syrup, fruit, or whipped cream."
}

# Save to JSON file
food_descriptions_json = os.path.join(food_vision_demo_path, 'food_descriptions.json')
with open(food_descriptions_json, 'w') as f:
    json.dump(food_descriptions, f, indent=4)

# Open JSON file
# Load the food descriptions from the JSON file
with open(food_descriptions_json, 'r') as f:
    food_descriptions = json.load(f)


In [6]:
class_names = list(food_descriptions.keys())
class_names

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [ ]:
%%writefile ../demo/model.py
import torch
import torchvision

range(50)

In [ ]:
# Define dataset directory
target_dir_name = f"../data/food-101_100_percent"

# Setup training and test directories
target_dir = Path(target_dir_name)
test_dir = target_dir / "test"
target_dir.mkdir(parents=True, exist_ok=True)


In [7]:
range(50)

range(0, 50)

In [16]:
p=range(1, 51)
p[0], p[-1]

(1, 50)

In [10]:
p[-1]

49